<a href="https://colab.research.google.com/github/puaqieshang/automatic-labeling-heart-vessels/blob/master/final_cnn_v7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Update from Previous Version

* Part 1 - Overfitting done
* Part 2 - Regularization done?? Val Accuracy > Train Accuracy tho 
* Part 3 - Tuned hyperparameters; rationale is in book

Results:

*   Test accuracy is promising
    *  Tested on 2 datasets 
    *  Accuracy is around 88%




In [0]:
# Import dependencies

import os
import glob
import time
import helper
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from collections import defaultdict

import torch
from torch import tensor, nn, optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")


# Data Retrieval


<table>
  <tr>
    <th>Label</th>
    <th>Class (Vessels)</th>
  </tr>
  <tr>
    <td>0</td>
    <td>LAD</td>
  </tr>
  <tr>
    <td>1</td>
    <td>Diagonals</td>
  </tr>
    <tr>
    <td>2</td>
    <td>Septals</td>
  </tr>
    <tr>
    <td>3</td>
    <td>LCX</td>
  </tr>
    <tr>
    <td>4</td>
    <td>Obtuse Marginal</td>
  </tr>
    <tr>
    <td>5</td>
    <td>Atrials</td>
  </tr>
    <tr>
    <td>6</td>
    <td>LCIM</td>
  </tr>
    <tr>
    <td>7</td>
    <td>Acutes</td>
  </tr>
    <tr>
    <td>8</td>
    <td>Crux</td>
  </tr>
</table>

## Import from Google Drive

In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')
# !unzip -uq "/content/drive/My Drive/ldai-load.zip"
!unzip -uq "/content/drive/My Drive/processed_v2.zip"
!ls "/content/" # Lists all the files 

In [0]:
def storeData(dest):

    X_all = []
    y_all = []
    folders = os.listdir(dest)
    idCount = 0

    for folder in folders:

        if (folder == ".DS_Store"):
            continue
        
        file_des = f"{dest}/{folder}"
        all_files = os.listdir(file_des)
        # print(f"Folder {folder} containes {all_files}")

        os.chdir(file_des)
        for f in glob.glob("*.txt"):
            
            geometry = np.genfromtxt(f)
            geometry = geometry.tolist()
            # print(f"The file {f} has {geometry[0]}")              
            # print(geometry.shape)
            X_all.append(geometry)
            y_all.append(folder)

            idCount = idCount +1

    X_all, y_all = shuffle(np.array(X_all), np.array(y_all), random_state=0)
    
    return X_all, y_all

dest = '/content/processed_v2/'
X_all, y_all = storeData(dest)


print(X_all.shape)
print(y_all[0])

## Develop Custom Dataset

In [0]:
class HeartVesselsDataset(Dataset):

    # Initialize your data, download, etc.
    def __init__(self, attributes_data, labels):

        self.x_data = attributes_data # Access dict
        self.y_data = labels # Access dict
        self.len = labels.shape[0]
        
    # USE DICTIONARY
    def __getitem__(self, index):
        # print(index)
        geometry = self.x_data[index]
        label = self.y_data[index] # Its a number
        # print(label)
        
        return geometry, tensor(int(label))


    def __len__(self):
        return len(self.x_data)



## Create Train and Validate Datasets

Cross validation folds

In [0]:
from sklearn.model_selection import KFold # import KFold
X = X_all 
print(X.shape)# create an array
y = y_all # Create another array
kf = KFold(n_splits=5,random_state=None, shuffle=False) # Define the split - into 5 folds 
# kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf) 
# kf = KFold(n_splits=5, random_state=None, shuffle=False)

for train_index, val_index in kf.split(X):
    
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]
    # print(X_val[0][0])



In [0]:

train_dataset = HeartVesselsDataset(X_train, y_train)
trainloader = DataLoader(dataset=train_dataset, batch_size=16,shuffle=True,num_workers=0)


val_dataset = HeartVesselsDataset(X_val, y_val)
valloader = DataLoader(dataset=val_dataset, batch_size=16,shuffle=True,num_workers=0)

print(f"Train has {len(trainloader)} batches and {train_dataset.len} datasets")
print(f"Validation has {len(valloader)} batches and {val_dataset.len} datasets")


## Inspecting Input - A Single Branch

In [0]:
geometries, label = next(iter(trainloader))

# print(f"Geo is of type {geometries.type}")
# print(f"Label is of type {label.type}")
print("A Random Coronary Branch")
print(f"Dimension of a branch/matrix is {geometries[0].shape}")

plt.imshow(geometries[0].view(20, 20))
plt.show()
print(f"The label is {label[0]}")

# Model Definition

So that different model architectures can work interchangeable



## Linear Layers

### Model 1 - nn.Linear in Sequential Form

A simple Multi-Layer Perceptron, but achieves accuracy of 0.78 - 0.8 after 350 epochs

In [0]:
# model = nn.Sequential(nn.Linear(400,128), nn.ReLU(), nn.Linear(128,64), 
#                       nn.ReLU(), nn.Linear(64,9), nn.LogSoftmax(dim=1))

# model = model.to(device)

### Model 2 - nn.Linear in Class form 
Basically same as previous model, just that its in nn.fucntional() form. This allows user to tune the parameters more, which is better. 


In [0]:
# # Start Timer
# t0 = time.time()


# class ConvNN(nn.Module):
#     def __init__(self):
#         super().__init__()
#         # Defining the layers, 128, 64, 9 units each
#         self.fc1 = nn.Linear(400, 128)
#         self.fc2 = nn.Linear(128, 64)
#         # Output layer, 9 units - one for each digit
#         self.fc3 = nn.Linear(64, 9)
        
#     def forward(self, x):
#         ''' Forward pass through the network, returns the output logits '''
        
#         x = self.fc1(x)
#         x = F.relu(x)
#         x = self.fc2(x)
#         x = F.relu(x)
#         x = self.fc3(x)
#         x = F.log_softmax(x, dim=1)
        
#         return x


# model = ConvNN()
# model = model.to(device)

## Convolutional Layers

Suggestions to Improve:
* Use He Initialization for ReLU layers instead of random or zero initiliazation. Refer to roadmap
* Use more regularization techniques (if necessary)

Things to note:
* Increasing number of layers by more than 2 do not improve the accuracy - could be due to my model being simple?
* No need to have a lot of nodes in a hidden layer; the input is not a 2D image anyways.
* [Dropout Layers FAQ ](https://https://stats.stackexchange.com/questions/240305/where-should-i-place-dropout-layers-in-a-neural-network )
* Dropout layers may result in [Test Accuracy > Train Accuracy](https://www.quora.com/How-can-I-explain-the-fact-that-test-accuracy-is-much-higher-than-train-accuracy)  

### Model 3 - Conv1D with 3 Hidden Layers

Comment out the Regularisation if not needed



In [0]:
class ConvNN(nn.Module):
    def __init__(self):
        super().__init__()
        
        # Input Tensor [batch, in_channels, in_width]
        x = torch.randn(100, 4).view(-1, 4, 100)
        # print(x.shape)

        # Defining the layers, 4, 8, 16 and 32 units each

        self.conv1 = nn.Conv1d(in_channels=4, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=64, kernel_size=3)
        self.conv3 = nn.Conv1d(in_channels=64, out_channels=256, kernel_size=3)

        # Activated during eval mode
        self.conv1_bn = nn.BatchNorm1d(16)
        self.conv2_bn = nn.BatchNorm1d(64)
        self.conv3_bn = nn.BatchNorm1d(256)
        self.fc1_bn = nn.BatchNorm1d(128)
        self.fc2_bn = nn.BatchNorm1d(32)
    
        self.fc_dropout = nn.Dropout(p=0.5)
        self.conv_dropout = nn.Dropout(p=0.1)
        
        self._to_linear = None

        self.convs(x)

        # self.fc1 = nn.Linear(self._to_linear, 64) #flattening.
        # self.fc2 = nn.Linear(64, 9) # 64 in, 9out bc we're doing 9 classes 

        self.fc1 = nn.Linear(self._to_linear, 128) #flattening.
        self.fc2 = nn.Linear(128, 32)
        self.fc3 = nn.Linear(32, 9)

    def convs(self, x):
        # max pooling over 2
        
        x = self.conv1(x)
        x = self.conv1_bn(x)  
        x = F.relu(x)
        x = self.conv_dropout(x)    
        x = F.max_pool1d(x, 2) #or Elu

        x = self.conv2(x)
        x = self.conv2_bn(x)   
        x = F.relu(x)
        x = self.conv_dropout(x)
        x = F.max_pool1d(x, 2) #or Elu

        x = self.conv3(x)
        x = self.conv3_bn(x)   
        x = F.relu(x)
        x = self.conv_dropout(x)
        x = F.max_pool1d(x, 2) #or Elu
        
        if self._to_linear is None:
            self._to_linear = x[0].shape[0]*x[0].shape[1]
            
        return x
        
    def forward(self, x):
        ''' Forward pass through the network, returns the output logits '''
        
        x = self.convs(x)
        x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 

        x = self.fc1(x)
        x = self.fc1_bn(x)
        x = F.relu(x)
        x = self.fc_dropout(x) 

        x = self.fc2(x) # bc this is our output layer. No activation here.
        x = self.fc2_bn(x)
        x = F.relu(x)
        x = self.fc_dropout(x)   

        x = self.fc3(x)

        return F.log_softmax(x, dim=1)
        
        return x


### Model 4 - Conv1D with 2 Layers


In [0]:
# class ConvNN(nn.Module):
#     def __init__(self):
#         super().__init__()
        
#         # Input Tensor [batch, in_channels, in_width]
#         x = torch.randn(100, 4).view(-1, 4, 100)
#         print(x.shape)
    
#         '''
#         The node layers and numbers are based on multiple websites
#         https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw
#         https://www.researchgate.net/post/In_neural_networks_model_which_number_of_hidden_units_to_select
#         '''
        
#         self.conv1 = nn.Conv1d(in_channels=4, out_channels=7, kernel_size=3) 
#         self.conv2 = nn.Conv1d(in_channels=7, out_channels=5, kernel_size=3)
#         # self.conv1_bn = nn.BatchNorm1d(7)
#         # self.conv2_bn = nn.BatchNorm1d(5)
#         self.fc_dropout = nn.Dropout(p=0.5)
#         self.conv_dropout = nn.Dropout(p=0.1)
        
#         self._to_linear = None

#         self.convs(x)

#         self.fc1 = nn.Linear(self._to_linear, 64) #flattening.
#         self.fc2 = nn.Linear(64, 9) # 64 in, 9out bc we're doing 9 classes 

#     def convs(self, x):
#         # max pooling over 2
#         x = self.conv1(x)
#         # x = self.conv1_bn(x)       
#         x = F.max_pool1d(self.conv_dropout(F.relu(x)), 2) #or Elu

#         x = self.conv2(x)
#         # x = self.conv2_bn(x)       
#         x = F.max_pool1d(self.conv_dropout(F.relu(x)), 2) #or Elu

#         # x = self.conv3(x)
#         # # x = self.conv3_bn(x)       
#         # x = F.max_pool1d(F.relu(x), 2) #or Elu
        
#         if self._to_linear is None:
#             self._to_linear = x[0].shape[0]*x[0].shape[1]
            
#         return x
        
#     def forward(self, x):
#         ''' Forward pass through the network, returns the output logits '''
        
#         x = self.convs(x)
#         x = x.view(-1, self._to_linear)  # .view is reshape ... this flattens X before 

#         x = self.fc1(x)
#         # x = self.fc1_bn(x)
#         x = F.relu(x)
#         x = self.fc_dropout(x) 

#         x = self.fc2(x) # bc this is our output layer. No activation here.
#         return F.log_softmax(x, dim=1)
        
#         return x


# Train and Validate Model




```
model.train(), model.eval()
```

Both are the same. By default all the modules are initialized to train mode (self.training = True). 

In case you want to validate your data, call model.eval() before feeding the data, as this will change the behavior of the BatchNorm (or Dropout) layer to use the running estimates instead of calculating them for the current batch.
If you want to train your model and can’t use a bigger batch size, you could switch e.g. to InstanceNorm.

In [0]:
def train(loader): # for train cases
    running_loss = 0
    sumCorrect = 0
    sumTotal = 0      

    model.train() # for train cases
        
    for X, y in loader:

        X = X.view(-1, 4, 100)
        # print(X.shape)
        X, y = X.to(device), y.to(device)
        
        optimiser.zero_grad()
        
        # train_X = train_X.view(-1, 4, 100)
        output = model(X.float())
        _, predicted = torch.max(output.data, 1)
        loss = criterion(output,y)

        loss.backward()
        optimiser.step()
        
        running_loss += loss.item()
        
        # Accuracy
        sumCorrect += (predicted == y).sum().item()
        sumTotal += y.size(0)

    l = running_loss/len(loader) #average loss for whole loader dataset
    acc = sumCorrect/sumTotal

    return acc, l

def validate(loader):
    running_loss = 0
    sumCorrect = 0
    sumTotal = 0      

    model.eval() # for validation cases
    with torch.no_grad():   
        for X, y in loader:

            X = X.view(-1, 4, 100)
            # print(X.shape)
            X, y = X.to(device), y.to(device)
            
            
            # train_X = train_X.view(-1, 4, 100)
            output = model(X.float())
            _, predicted = torch.max(output.data, 1)
            loss = criterion(output,y)

            running_loss += loss.item()         

            # Accuracy
            sumCorrect += (predicted == y).sum().item()
            sumTotal += y.size(0)
            # print(sumTotal)   

    l = running_loss/len(loader) #average loss for whole loader dataset
    acc = sumCorrect/sumTotal

    return acc, l

def train_and_validate(trainloader, valloader):
    
    for e in tqdm(range(EPOCHS), position=0, leave=True):

        # train
        train_acc, train_loss = train(trainloader)
        train_acc_list.append(train_acc)
        train_loss_list.append(train_loss)
        
        # Validate
    
        val_acc, val_loss = validate(valloader)
        val_acc_list.append(val_acc)
        val_loss_list.append(val_loss)

        # print(f"Epoch {e} has test accuracy of {round(test_acc, 3)} and loss of {round(test_loss,3)}")
    
    return sum(val_acc_list)/len(val_acc_list)

In [0]:
model = ConvNN()
model = model.to(device)

## Calculate Accuracy and Loss

In [0]:
# Start Timer
t0 = time.time()

'''
https://medium.com/octavian-ai/which-optimizer-and-learning-rate-should-i-use-for-deep-learning-5acb418f9b2
'''
criterion = nn.NLLLoss()
# optimiser = optim.Adam(model.parameters(), lr=0.0001)
optimiser = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

# Refer to Deep Learning Roadmap

train_acc_list, train_loss_list, val_acc_list, val_loss_list = [],[],[],[]
EPOCHS = 300

avg_val_accuracy = train_and_validate(trainloader, valloader)

t1 = time.time()
duration = (t1-t0)/60
print("\nTime taken to train is " + "{:.2f}".format(duration) + " minutes.")

## Plot Accuracies and Loss vs Epochs

In [0]:
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd

style.use("ggplot")

def create_acc_loss_graph():
    epoch = list(range(0, EPOCHS, 1))

    d = {'epochs': epoch,
        'train_acc': train_acc_list,
        'train_loss': train_loss_list,
        'val_acc': val_acc_list, 
        'val_loss': val_loss_list}

    df = pd.DataFrame(d)

    df['train_acc_avg'] = df['train_acc'].ewm(alpha=.02).mean()  # exponential weighted moving average
    df['val_acc_avg'] = df['val_acc'].ewm(alpha=.02).mean()
    df['train_loss_avg'] = df['train_loss'].ewm(alpha=.02).mean()
    df['val_loss_avg'] = df['val_loss'].ewm(alpha=.02).mean()

    # Then plot using pandas:
    df.plot(x='epochs', y=['train_acc_avg', 'val_acc_avg'], figsize=(8,4))
    plt.ylabel("Accuracy")
    df.plot(x='epochs', y=['train_loss_avg', 'val_loss_avg'], figsize=(8,4))
    plt.ylabel("Loss")

    plt.show()

create_acc_loss_graph()

## Displays Model Summary

In [0]:
from torchvision import models
model = models.vgg16()
print(model)

In [0]:
# Prints the model's state for each layer
print("Model's state_dict:")
for param_tensor in model.state_dict():
    print(param_tensor, "\t", model.state_dict()[param_tensor].size())

# Print the weights of the model - A LOT OF MATRICES
# print("Optimizer's state_dict:")
# for var_name in optimiser.state_dict():
#     print(var_name, "\t", optimiser.state_dict()[var_name])

# Visualization



In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

dataiter = iter(valloader)
geometries, labels = dataiter.next()
geom = geometries[0]
# Convert 2D matrix/image to 1D vector

geom = geom.resize_(1, 400)


# TODO: Calculate the class probabilities (softmax) for img
with torch.no_grad():
    
    logps = model(geom.cuda().view(-1, 4, 100).float()) # convert to cuda tensor to run faster
    
ps = torch.exp(logps)

# Plot the image and probabilities

def view_classify(geom, ps):
    ''' Function for viewing an image and it's predicted classes.
    '''
    ps = ps.data.cpu().numpy().squeeze()

    fig, (ax1, ax2) = plt.subplots(figsize=(6,9), ncols=2)
    ax1.imshow(geom.resize_(20, 20).numpy().squeeze()) #First figure on left
    
    ax1.axis('off')
    ax2.barh(np.arange(9), ps) #Make a horizontal bar plot
    ax2.set_aspect(0.1)
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(vessels_names, size='medium');
    ax2.set_title('Class Probability')
    ax2.set_xlim(0, 1.1)

    plt.tight_layout()


geom_numpy = geom.cpu().numpy()
view_classify(geom.reshape(20, 20), ps) # convert back to numpy

print(f"The actual vessel is {vessels_names[labels[0]]}")


# Analysis 

## Confusion Matrix

In [0]:
from sklearn import metrics

# True values
y_true = true_class_array
# Predicted values
y_pred = predicted_class_array

# Print the confusion matrix
print(metrics.confusion_matrix(y_true, y_pred))


In [0]:
# Print the precision and recall, among other metrics
print(metrics.classification_report(y_true, y_pred, digits=3))

# Main Pipeline

Done **last** after finalising code.

Basically change every module or header section into a def so that its easier to call. 

All the user does is to run this code.

 Outputs the average accuracy of the model:

In [0]:

accuracies = []
import statistics as stats


X = X_all 
y = y_all # Create another array
kf = KFold(n_splits=5,random_state=None, shuffle=False) # Define the split - into 5 folds 
# kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
# print(kf) 
# kf = KFold(n_splits=5, random_state=None, shuffle=False)

for train_index, val_index in kf.split(X):
    # print("TRAIN:", train_index, "TEST:", val_index)
    # print(train)
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    train_dataset = HeartVesselsDataset(X_train, y_train)
    trainloader = DataLoader(dataset=train_dataset, batch_size=16,shuffle=True,num_workers=0)

    val_dataset = HeartVesselsDataset(X_val, y_val)
    valloader = DataLoader(dataset=val_dataset, batch_size=16,shuffle=True,num_workers=0)

    model = ConvNN()
    model = model.to(device)

    t0 = time.time()
    criterion = nn.NLLLoss()
    optimiser = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-4)

    # Refer to Deep Learning Roadmap
    train_acc_list, train_loss_list, val_acc_list, val_loss_list = [],[],[],[]
    EPOCHS = 275

    acc = train_and_validate(trainloader, valloader)
    accuracies.append(acc)

    t1 = time.time()
    duration = (t1-t0)/60
    print("\nTime taken to train for epoch is " + "{:.2f}".format(duration) + " minutes.")



print(f"\nThis model has an accuracy of {round(stats.mean(accuracies), 3)} +/- {round(stats.stdev(accuracies), 3)}")      

# Test Model on Unseen Data 
Well technically, `model.eval() `is not learning any new weights so its testing

In [0]:
!unzip -uq "/content/drive/My Drive/processed-unseen.zip" -d '/content' # sends to the specific directory

dest = '/content/processed-unseen/'
X_test, y_test = storeData(dest)

test_dataset = HeartVesselsDataset(X_test, y_test)
testloader = DataLoader(dataset=test_dataset, batch_size=16,shuffle=True,num_workers=0)

In [0]:
vessels_names = ['ldai', 'diags', 'septals', 'lcxi', 'obtmar', 'atrials', 'lcim', 'acutes', 'crux']

def test(model):

    correct = 0
    total = 0
    vessels_count = np.zeros((9,), dtype=int)
    vessels_correct = np.zeros((9,), dtype=int)
    mainBifurcations_count = np.zeros((3,), dtype=int) 
    mainBifurcations_correct = np.zeros((3,), dtype=int) 

    predicted_class_array = []
    true_class_array = []

    with torch.no_grad():

        model.eval()

        for test_X, test_y in testloader:

            test_y_list = test_y.tolist()
            # true_class_array.append(test_y)
            true_class_array.extend(test_y_list)
            test_X, test_y = test_X.to(device), test_y.to(device)

            for i in tqdm(range(len(test_X)), position=0, leave=True):
                real_class = test_y[i].item()
                # print(f"real class is {real_class}")

                vessels_count[real_class] += 1
                # ps = net(test_X[i].view(-1, 1, 20, 20))[0] 
                geom = test_X[i]
                geom = geom.view(-1, 4, 100)
                    
                logps = model(geom.float())
                ps = torch.exp(logps)  #probabilities
                predicted_class = torch.argmax(ps).item()
                predicted_class_array.append(predicted_class)

                if predicted_class is real_class:
                    vessels_correct[real_class] += 1
                    correct += 1
                
                else: 
                    # These numbers follow the sequence in the "vessels_name" list
                    # print(predicted_class)
                    if predicted_class in {0, 1, 2}: 
                        
                        if real_class in {0, 1, 2}:
                            # print(f"predicted {predicted_class}, real {real_class}")
                            mainBifurcations_correct[0] += 1

                    elif predicted_class in {3, 4}:
                        if real_class in {3, 4}:
                            # print(f"predicted {predicted_class}, real {real_class}")
                            mainBifurcations_correct[1] += 1
                    
                    elif predicted_class in {7, 8}:
                        if real_class in {7, 8}:
                            mainBifurcations_correct[2] += 1
                    else: 
                        continue

                total += 1

    print("\n\nOverall Accuracy is:", round(correct/total,3))
    print("------------------------")

    for i in range(len(vessels_count)):
        acc = round(vessels_correct[i]/vessels_count[i],3)
        string = f"Accuracy of {vessels_names[i]} is:"
        print("{:<30} {:<5}".format(string, acc))

    print("------------------------")
    print(f"No of samples for each vessel in a batch is {vessels_count}")

    mainBifurcations_count[0] = vessels_count[0] + vessels_count[1] + vessels_count[2] #LCA
    mainBifurcations_count[1] = vessels_count[3] + vessels_count[4] #LCX
    mainBifurcations_count[2] = vessels_count[7] + vessels_count[8] #RCA 

    mainBifurcations_correct[0] += vessels_correct[0] + vessels_correct[1] + vessels_correct[2]  #LCA
    mainBifurcations_correct[1] += vessels_correct[3] + vessels_correct[4]  #LCX
    mainBifurcations_correct[2] += vessels_correct[7] + vessels_correct[8]  #RCA

    print("------------------------")
    print(f"Bifurcation LCA has accuracy of {round(mainBifurcations_correct[0]/mainBifurcations_count[0], 3)}")
    print(f"Bifurcation LCX has accuracy of {round(mainBifurcations_correct[1]/mainBifurcations_count[1], 3)}")
    print(f"Bifurcation RCA has accuracy of {round(mainBifurcations_correct[2]/mainBifurcations_count[2], 3)}")

    # print(test_y)
    return true_class_array, predicted_class_array


true_class_array, predicted_class_array = test(model)
